# 

In [531]:
import numpy as np
from math import sin, cos, radians
import random
from PIL import Image
from PIL import ImageOps

In [532]:
def process_obj(filename : str):    
    with open(filename) as obj:
        obj.seek(0, 2)
        length = obj.tell()
        obj.seek(0, 0)

        vertices = []
        vertices_texture = []
        vertices_normal = []
        faces = []
        
        while obj.tell() != length:
            line = obj.readline().split()

            if len(line) == 0:
                continue
            
            if line[0] == 'v':
                vertices.append(list(map(float, line[1:])))
            elif line[0] == 'vt':
                vertices_texture.append(list(map(float, line[1:])))
            elif line[0] == 'vn':
                vertices_normal.append(list(map(float, line[1:])))
            elif line[0] == 'f':
                faces.append([[int(num) - 1 for num in vertex.split('/')] for vertex in line[1:]])
        return np.array(vertices), np.array(vertices_texture), np.array(vertices_normal), np.array(faces)

In [533]:
def barycentric_coordinates(i, j, x, y):
    system = np.vstack((np.ones(3), x, y))
    column = np.array([1, i, j])
    
    return np.linalg.solve(system, column)

In [577]:
def draw_triangle_gouraud(image, v, face, w, h, texture, vt, l, vn, z_buffer):
    polygon = face[:, 0]
    normals = face[:, 2]
    
    x = np.array([vertices[polygon[i]][0] for i in range(len(polygon))])
    y = np.array([vertices[polygon[i]][1] for i in range(len(polygon))])
    z = np.array([vertices[polygon[i]][2] for i in range(len(polygon))])
    
    xmin = max(int(min(x)), 0)
    xmax = min(int(max(x)), w - 1)
    ymin = max(int(min(y)), 0)
    ymax = min(int(max(y)), w - 1)

    uv = np.array([[vt[face[j, 1]][i] for j in range(len(polygon))] for i in range(2)])

    I = np.array([min(np.dot(vn[normals[i]], l)/(np.linalg.norm(vn[normals[i]]) * np.linalg.norm(l)), 0) for i in range(len(polygon))])

    for i in range(xmin, xmax + 1):
        for j in range(ymin, ymax + 1):
            if len(polygon) == 3:
                coords = barycentric_coordinates(i, j, x, y)
                z_coord = np.dot(coords, z)
                I3 = I
            elif len(polygon) == 4:
                coords = barycentric_coordinates(i, j, x[:3], y[:3])
                z_coord = np.dot(coords, z[:3])
                I3 = I[:3]
                uv3 = uv[:, [0,1,2]]
                if (coords < 0).any():
                    coords = barycentric_coordinates(i, j, np.array([x[0], x[3], x[2]]), np.array([y[0], y[3], y[2]]))
                    z_coord = np.dot(coords, np.array([z[0], z[3], z[2]]))
                    I3 = np.array([I[0], I[3], I[2]])
                    uv3 = uv[:, [0,3,2]]

            if z_coord < z_buffer[i, j] and (coords >= 0).all():
                z_buffer[i, j] = z_coord
                coef = np.dot(coords, I3)
                pixel_index = [round(texture.shape[1] * np.dot(coords, uv3[i])) for i in range(2)]
                color = texture[pixel_index[1]][pixel_index[0]]
                image[i, j] = color * -coef

In [524]:
def draw_triangle_phong(image, vertices, face, w, h, texture, vertices_texture, l, vertices_normal, z_buffer):
    triangle = face[:, 0]
    normals = face[:, 2]
    x0 = vertices[triangle[0]][0]
    y0 = vertices[triangle[0]][1]
    x1 = vertices[triangle[1]][0]
    y1 = vertices[triangle[1]][1]
    x2 = vertices[triangle[2]][0]
    y2 = vertices[triangle[2]][1]
    
    xmin = int(min(x0, x1, x2))
    xmax = int(max(x0, x1, x2))
    ymin = int(min(y0, y1, y2))
    ymax = int(max(y0, y1, y2))

    xmin = 0 if xmin < 0 else xmin
    ymin = 0 if ymin < 0 else ymin
    xmax = xmax if xmax < w - 1 else w - 1
    ymax = ymax if ymax < h - 1 else h - 1
    
    ut0 = vertices_texture[face[0, 1]][0]
    ut1 = vertices_texture[face[1, 1]][0]
    ut2 = vertices_texture[face[2, 1]][0]
    vt0 = vertices_texture[face[0, 1]][1]
    vt1 = vertices_texture[face[1, 1]][1]
    vt2 = vertices_texture[face[2, 1]][1]

    for i in range(xmin, xmax + 1):
        for j in range(ymin, ymax + 1):
            coords = barycentric_coordinates(i, j, x0, y0, x1, y1, x2, y2)
            z_coord = coords[0] * vertices[triangle[0]][2] + coords[1] * vertices[triangle[1]][2] + coords[2] * vertices[triangle[2]][2]
            
            if z_coord < z_buffer[i, j]:
                if (coords is not None) and coords[0]>=0 and coords[1]>=0 and coords[2]>=0:
                    pixel_normal = vertices_normal[normals[0]] * coords[0] + vertices_normal[normals[1]] * coords[1] + vertices_normal[normals[2]] * coords[2] 
                    z_buffer[i, j] = z_coord
                    coef = min(np.dot(pixel_normal, l)/(np.linalg.norm(pixel_normal) * np.linalg.norm(l)), 0)
                    color = texture[round(texture.shape[1] * (coords[0] * vt0 + coords[1] * vt1 + coords[2] * vt2))][round(texture.shape[1] * (coords[0] * ut0 + coords[1] * ut1 + coords[2] * ut2))]
                    image[i, j] = color * -coef

In [552]:
def find_all_normals(vertices, faces):
    faces_normal = []
    for face in faces:
        polygon = face[:, 0]
        vec1 = vertices[polygon[1]] - vertices[polygon[0]]
        vec2 = vertices[polygon[2]] - vertices[polygon[0]]
        faces_normal.append(np.cross(vec1, vec2))

    return np.array(faces_normal)

In [553]:
def find_all_faces_for_vertex(faces):
    vertices_to_faces = {}

    i = 0
    for face in faces:
        triangle = face[:, 1]
        for v in triangle:
            vertices_to_faces.setdefault(v, []).append(i)
        i += 1
    return vertices_to_faces    

In [554]:
def interpolate_vertex_normals(vertice_in_faces, faces_normal):
    vertices_normal = {}
    for vertex, faces in vertice_in_faces.items():
        vector_sum = np.zeros(shape=3,)
        for i in faces:
            vector_sum += faces_normal[i]
        vertices_normal[vertex] = vector_sum/np.linalg.norm(vector_sum)
    return vertices_normal

In [603]:
degrees = {'x' : 180, 'y' : -90, 'z' : 90}
a = {k: radians(v) for k,v in degrees.items()}

height = 1000
width = 1000

init_scale = 100
model_shift = np.array([500 / init_scale, 0, 0])
white = np.array([-255, -255, -255])
l = np.array([0, 0, 1])

X_rotate = np.array([[1, 0, 0], [0, cos(a['x']), -sin(a['x'])], [0, sin(a['x']), cos(a['x'])]])
Y_rotate = np.array([[cos(a['y']), 0, sin(a['y'])], [0, 1, 0], [-sin(a['y']), 0, cos(a['y'])]])
Z_rotate = np.array([[cos(a['z']), -sin(a['z']), 0], [sin(a['z']), cos(a['z']), 0], [0, 0, 1]])

In [604]:
filename = 'frog_model.obj'
texture_filename = 'frog.jpg'
with Image.open(texture_filename) as uv:
    img = uv.convert('RGB')
    img = ImageOps.flip(img)
    texture = np.array(img)
        
vertices, vertices_texture, vertices_normal_auto, faces = process_obj(filename)
vertice_in_faces = find_all_faces_for_vertex(faces)
faces_normal = find_all_normals(vertices, faces)
vertices_normal_manual = interpolate_vertex_normals(vertice_in_faces, faces_normal)

R = X_rotate @ Y_rotate @ Z_rotate
vertices = (R @ vertices.T).T
vertices_normal_auto = (R @ vertices_normal_auto.T).T
faces_normal = (R @ faces_normal.T).T
vertices_normal_manual = (R @ vertices_normal_auto.T).T
light = (R @ np.array([0, 0.1, 0.1]).T).T

vertices += model_shift

z_shift = 200.0 * abs(vertices[:, 2].min())
scale = init_scale * z_shift

vertices[:, 0] = scale * vertices[:, 0] / (vertices[:, 2] + z_shift) + height / 2
vertices[:, 1] = scale * vertices[:, 1] / (vertices[:, 2] + z_shift) + width / 2
vertices[:, 2] *= init_scale

matrix = np.full(shape=(height, width, 3), fill_value=[0, 0, 0], dtype = np.uint8)
z_buffer = np.full(shape=(height, width), fill_value=np.inf)

In [605]:
i = 0
for face in faces:
    norm = faces_normal[i]
    angle = np.dot(norm, l)/np.linalg.norm(norm)
    if (angle < 0):
        draw_triangle_gouraud(matrix, vertices, face, height, width, texture, vertices_texture, light, vertices_normal_manual, z_buffer)
    i += 1    
Image.fromarray(matrix, 'RGB').save("draw_frog.jpg")